In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from typing import Optional

In [ ]:
base_url= "https://www.berlin.de"
faq = "/einwanderung/service/faq/"


In [ ]:
r = requests.get(base_url + faq)

In [ ]:
print(r.text)

In [ ]:
html = BeautifulSoup(r.text, "html.parser")

In [ ]:
def get_content(page:BeautifulSoup, tag:str = None, keyword:str = "", **kwargs):
    
    tags = page.find_all(tag, **kwargs)
    if tag == "a":
        response = []
        for element in tags:
            if element.text.find(keyword) != -1:
                content = {
                    "content" : element.text,
                    "address" : base_url + element["href"]
                }
                response.append(content)
        return response
    elif tag == "div":
        for element in tags:
            return element
            # if element.text.find("textile") != -1:
            #     print(element.text)

def drop_content(content:BeautifulSoup, regex:str, drop:str)-> Optional[BeautifulSoup]:
    # drop = prev or following
    
    # hits = re.search("<h.*>.*Für Sie zuständig", str(content), re.IGNORECASE)
    hits = re.search(regex, str(content), re.IGNORECASE).group()
    
    if not hits:
        return
    
    if drop == "prev":
        pass

    if drop == "following":
        element = content.find(string= "Für Sie zuständig")
        if element:
            parent = element.parent
            for sibling in parent.find_all_next():
                sibling.extract()
            element.parent.decompose()
    return content
                

In [ ]:
# im Falle der Seiten von Berlin.de 
main_content = get_content(html, "div", id = "layout-grid__area--maincontent")

# print(main_content)

keywordsearch = get_content(main_content, "a", "Aufenthaltserlaubnis")

# print(keywordsearch)
# print("\n\n" + "#"*30)

for response in keywordsearch:
    req = requests.get(response["address"])
    html_con = BeautifulSoup(req.text, "html.parser")
    # print(req.text)
    textcontent = get_content(html_con, tag = "div", id = "layout-grid__area--maincontent")
    # print(textcontent.text)
    # print("\n\n" + "#"*30)

    furtherlinks = get_content(textcontent, "a")
    # print(furtherlinks)
    # print("\n\n" + "#"*30)



for response in furtherlinks:
    req = requests.get(response["address"])
    html_con = BeautifulSoup(req.text, "html.parser")
    # print(req.text)
    textcontent = get_content(html_con, tag = "div", id = "layout-grid__area--maincontent")
    print(drop_content(textcontent, regex= "<h.*>.*Für Sie zuständig",drop = "following").get_text())
    # print("\n\n" + "#"*30)

In [ ]:
re.search("<h.*>.*Für Sie zuständig", str(textcontent)).group()